# Module Import

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os 
import random

import matplotlib.font_manager as fm

from scipy import stats
from datetime import datetime
# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'NanumGothic' # 한글깨짐 방지

# Data Load

In [7]:
survey_df = pd.read_excel('./../dataset/survey_data/학생안전사고예방을 위한 설문(응답).xlsx', index_col = 0)
survey_df.head()

,1. 귀하의 성별,2. 귀하의 출생년도,3. 귀하는 어떤 활동을 하면서 사고가 가장 많이 날 것 같나요?,"4. 귀하는 사고 발생 시, 신체의 어떤 부위를 가장 많이 다칠 것 같나요?","5. 귀하는 사고 발생 시, 어떤 시간에 가장 많이 다칠 것 같나요?","6. 귀하는 사고 발생 시, 어떻게 사고가 일어날 것 같나요?","7. 본인은 학생들이 가장 사고가 잘 일어나는 곳을 알고 있다면, 사고를 예방하는데 도움이 될 것 같나요?",9. 귀하는 학생 안전 교육의 필요성을 느끼고 있나요?,10. 귀하는 학생 사고 예방을 위해서 무엇이 가장 필요한 것 같나요?,8. 학교에서 얼마나 자주 안전교육을 실시하고 있나요?
타임스탬프,,,,,,,,,,
2024-07-16 22:50:35.547,남자,1999,구기운동,손,체육활동,물리적인 힘 노출,5,5,학생 사고 예방을 위해 정기적인 안전 교육이 필요하다고 생각.,특별한 날(학교 행사 등)에만
2024-07-16 23:02:32.525,여자,2010,장난/놀이,손,쉬는시간,사람과의 충돌,5,5,저는 학교에서 안전 수칙을 철저히 알려주세요.,분기 마다
2024-07-16 23:40:14.167,여자,2010,장난/놀이,팔,쉬는시간,사람과의 충돌,5,5,안전을 위해 학교 내 위험 구역 홍보?.,분기 마다
2024-07-16 23:43:14.177,여자,2012,"구기운동, 실험실습",손,체육활동,사람과의 충돌,4,4,사고를 줄이기 위해 교실마다 응급처치키트 배치,특별한 날(학교 행사 등)에만
2024-07-17 15:57:39.669,여자,2006,구기운동,다리,체육활동,염좌(삐임 등 신체충격),5,5,안전 교육 실시.,특별한 날(학교 행사 등)에만


# Preprocessing

In [8]:
# 1. 칼럼명 바꾸기
survey_df = survey_df.rename(columns={
    '1. 귀하의 성별' : '성별',
    '2. 귀하의 출생년도' : '출생년도',
    '3. 귀하는 어떤 활동을 하면서 사고가 가장 많이 날 것 같나요?' : '사고당시활동',
    '4. 귀하는 사고 발생 시, 신체의 어떤 부위를 가장 많이 다칠 것 같나요?' : '사고부위',
    '5. 귀하는 사고 발생 시, 어떤 시간에 가장 많이 다칠 것 같나요?' : '사고시간',
    '6. 귀하는 사고 발생 시, 어떻게 사고가 일어날 것 같나요?' : '사고형태', 
    '7. 본인은 학생들이 가장 사고가 잘 일어나는 곳을 알고 있다면, 사고를 예방하는데 도움이 될 것 같나요?' : '인지',
    '8. 학교에서 얼마나 자주 안전교육을 실시하고 있나요?' : '안전교육실시정도',
    '9. 귀하는 학생 안전 교육의 필요성을 느끼고 있나요?' : '안전교육필요성',
    '10. 귀하는 학생 사고 예방을 위해서 무엇이 가장 필요한 것 같나요?' : '피드백'
})


In [9]:
survey_df

,성별,출생년도,사고당시활동,사고부위,사고시간,사고형태,인지,안전교육필요성,피드백,안전교육실시정도
타임스탬프,,,,,,,,,,
2024-07-16 22:50:35.547,남자,1999,구기운동,손,체육활동,물리적인 힘 노출,5,5,학생 사고 예방을 위해 정기적인 안전 교육이 필요하다고 생각.,특별한 날(학교 행사 등)에만
2024-07-16 23:02:32.525,여자,2010,장난/놀이,손,쉬는시간,사람과의 충돌,5,5,저는 학교에서 안전 수칙을 철저히 알려주세요.,분기 마다
2024-07-16 23:40:14.167,여자,2010,장난/놀이,팔,쉬는시간,사람과의 충돌,5,5,안전을 위해 학교 내 위험 구역 홍보?.,분기 마다
2024-07-16 23:43:14.177,여자,2012,"구기운동, 실험실습",손,체육활동,사람과의 충돌,4,4,사고를 줄이기 위해 교실마다 응급처치키트 배치,특별한 날(학교 행사 등)에만
2024-07-17 15:57:39.669,여자,2006,구기운동,다리,체육활동,염좌(삐임 등 신체충격),5,5,안전 교육 실시.,특별한 날(학교 행사 등)에만
...,...,...,...,...,...,...,...,...,...,...
2024-07-22 20:34:31.161,남자,2011,실험실습,흉복부,휴식시간 및 청소시간,높은 곳에서 떨어짐,4,4,학교에서 안전 수칙을 잘 지킬 수 있도록 학생들에게 책임감을 심어주면 좋겠어요.,특별한 날(학교 행사 등)에만
2024-07-22 20:34:59.622,남자,2014,장난/놀이,머리(두부),쉬는시간,염좌(삐임 등 신체충격),4,4,안전한 학교 생활을 위해 학교 내 모든 구성원이 협력해야 한다고 생각해요.,특별한 날(학교 행사 등)에만
2024-07-22 20:35:15.980,여자,2014,장난/놀이,다리,체육활동,미끄러짐,4,4,사고 예방을 위해 학교 내 모든 시설을 정기적으로 점검하면 좋겠어요.,특별한 날(학교 행사 등)에만
